In [1]:
!gcloud config set project  bwh-midrc-rapid-res-1655321320

Updated property [core/project].


In [2]:
from google.colab import auth
auth.authenticate_user()

In [3]:
  project_name = "bwh-midrc-rapid-res-1655321320"
  bucket_name = "midrc-analysis-bwh"
  bucket_path = "bpr-results/"

In [4]:
#install nrrd
!pip install pynrrd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.7 MB 8.5 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6


In [5]:
!pip install SimpleITK

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 52.8 MB 13.8 MB/s 


In [6]:
!pip install pydicom

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 5.0 MB/s 


In [7]:
# Install BPR
import shutil
import os
if os.path.isdir('/content/BodyPartRegression'):
  shutil.rmtree('/content/BodyPartRegression')
!pip install torch==1.8.1 pytorch-lightning==1.2.10 torchtext==0.9.1 torchvision==0.9.1 torchaudio==0.8.1 dataclasses==0.6
!pip install bpreg
!git clone https://github.com/MIC-DKFZ/BodyPartRegression.git
# !pip install torch==1.8.1 pytorch-lightning==1.2.10 torchtext==0.9.1

import bpreg 
import seaborn as sb 

!pip install opencv-python-headless==4.1.2.30 # https://stackoverflow.com/questions/70537488/cannot-import-name-registermattype-from-cv2-cv2/70547274
from BodyPartRegression.docs.notebooks.utils import * 

# Get revision number for bpreg 
!pip show bpreg > '/content/bpreg_output.txt'
f = open('/content/bpreg_output.txt')
d = {}
for line in f:
  (key, val) = line.split(': ')
  d[key] = val
bpr_revision_number = d['Version'].strip()
print(bpr_revision_number)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 804.1 MB 2.3 kB/s 
     |████████████████████████████████| 841 kB 47.7 MB/s 
     |████████████████████████████████| 7.1 MB 44.4 MB/s 
     |████████████████████████████████| 17.4 MB 586 kB/s 
     |████████████████████████████████| 1.9 MB 49.2 MB/s 
     |████████████████████████████████| 176 kB 27.9 MB/s 
     |████████████████████████████████| 829 kB 58.5 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491068 sha256=d2295feddd7251a346b5b30c3515e322acc14aeb6c3de7bedb4ac25ca815a5f3
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
  Attempting uninstall: future
    Found

In [8]:
%%capture
!pip install dicomweb-client

In [9]:
import os
import glob
import numpy as np
import shutil
from google.cloud import storage
import nrrd
import SimpleITK as sitk
import json
import matplotlib.pyplot as plt
from matplotlib import transforms
from matplotlib.colors import ListedColormap
import csv
import pandas as pd
import pydicom

In [10]:
%%bigquery --project=bwh-midrc-rapid-res-1655321320 ct_limited_open_a1_r1

WITH
  nlst_instances_per_series AS (
    SELECT
      DISTINCT(StudyInstanceUID),
      SeriesInstanceUID,
      COUNT(DISTINCT(SOPInstanceUID)) AS num_instances,
      COUNT(DISTINCT(ARRAY_TO_STRING(ImagePositionPatient,"/"))) AS position_count,
      COUNT(DISTINCT(ARRAY_TO_STRING(ImageOrientationPatient,"/"))) AS orientation_count,
      MIN(SAFE_CAST(SliceThickness AS float64)) AS min_SliceThickness,
      MAX(SAFE_CAST(SliceThickness AS float64)) AS max_SliceThickness,
      MIN(SAFE_CAST(ImagePositionPatient[SAFE_OFFSET(2)] AS float64)) as min_SliceLocation, 
      MAX(SAFE_CAST(ImagePositionPatient[SAFE_OFFSET(2)] AS float64)) as max_SliceLocation,
      STRING_AGG(DISTINCT(SAFE_CAST("LOCALIZER" IN UNNEST(ImageType) AS string)),"") AS has_localizer
    FROM
      bwh-midrc-rapid-res-1655321320.midrc_dicom_us.dicom_all
    WHERE
      (collection_id = "Open-R1" or collection_id = "Open-A1") and Modality = "CT"
    GROUP BY
      StudyInstanceUID,
      SeriesInstanceUID
      ), 
  nlst_values_per_series AS (
    SELECT 
    ANY_VALUE(dicom_all.PatientID) AS PatientID,
    dicom_all.SeriesInstanceUID,
    ANY_VALUE(nlst_instances_per_series.num_instances) AS num_instances,
    ANY_VALUE(nlst_instances_per_series.max_SliceThickness) AS SliceThickness,
    ANY_VALUE((nlst_instances_per_series.max_SliceLocation - nlst_instances_per_series.min_SliceLocation)) AS PatientHeightScanned
  FROM
    bwh-midrc-rapid-res-1655321320.midrc_dicom_us.dicom_all AS dicom_all
  JOIN
    nlst_instances_per_series
  ON
    dicom_all.SeriesInstanceUID = nlst_instances_per_series.SeriesInstanceUID
  WHERE
    min_SliceThickness >= 1.5 
    AND max_SliceThickness <= 3.5 
    AND nlst_instances_per_series.num_instances > 100
    AND nlst_instances_per_series.num_instances/nlst_instances_per_series.position_count = 1
    AND nlst_instances_per_series.orientation_count = 1
    AND has_localizer = "false"
  GROUP BY
    SeriesInstanceUID
  )
  SELECT 
    dicom_all.PatientID,
    dicom_all.StudyInstanceUID,
    dicom_all.SeriesInstanceUID,
    dicom_all.SOPInstanceUID,
    dicom_all.collection_id,
    dicom_all.PatientAge,
    dicom_all.PatientWeight,
    nlst_values_per_series.num_instances,
    nlst_values_per_series.SliceThickness,
    nlst_values_per_series.PatientHeightScanned
  FROM
    bwh-midrc-rapid-res-1655321320.midrc_dicom_us.dicom_all AS dicom_all
  JOIN
    nlst_values_per_series 
  ON
    dicom_all.SeriesInstanceUID = nlst_values_per_series.SeriesInstanceUID

In [11]:
def download_series(study_instance_uid, series_instance_uid, sop_instance_uids, dest_dir):
  import pydicom
  token = !gcloud auth print-access-token
  token = token[0]

  PROJECT_ID="bwh-midrc-rapid-res-1655321320"
  REGION="us-central1"

  DATASET_ID="midrc"
  DICOM_STORE_ID="midrc-dicom"
  
  my_project = "bwh-midrc-rapid-res-1655321320"
  location = "us-central1"
  dataset_id = "midrc"
  dicom_store_id = "midrc-dicom"

  url = f"https://healthcare.googleapis.com/v1/projects/{my_project}/locations/{location}/datasets/{dataset_id}/dicomStores/{dicom_store_id}/dicomWeb"
  headers = {
      "Authorization" : "Bearer %s" % token
  }

  import dicomweb_client

  client = dicomweb_client.api.DICOMwebClient(url, headers=headers)

  idx=0
  for sop_instance_uid in sop_instance_uids:
    retrievedInstance = client.retrieve_instance(
                study_instance_uid=study_instance_uid,
                series_instance_uid=series_instance_uid,
                sop_instance_uid=sop_instance_uid)
    pydicom.filewriter.write_file(f"{dest_dir}/file{idx}.dcm", retrievedInstance)
    idx+=1

In [12]:
def file_exists_in_bucket(project_name, bucket_name, file_gs_uri):
  
  """
  Check whether a file exists in the specified Google Cloud Storage Bucket.

  Arguments:
    project_name : required - name of the GCP project.
    bucket_name  : required - name of the bucket (without gs://)
    file_gs_uri  : required - file GS URI
  
  Returns:
    file_exists : boolean variable, True if the file exists in the specified,
                  bucket, at the specified location; False if it doesn't.

  Outputs:
    This function [...]
  """

  storage_client = storage.Client(project = project_name)
  bucket = storage_client.get_bucket(bucket_name)
  
  bucket_gs_url = "gs://%s/"%(bucket_name)
  path_to_file_relative = file_gs_uri.split(bucket_gs_url)[-1]

  print("Searching %s for: \n%s\n"%(bucket_gs_url, path_to_file_relative))

  file_exists = bucket.blob(path_to_file_relative).exists(storage_client)
  
  return file_exists

In [13]:
import logging
urllib3_logger = logging.getLogger('urllib3')
urllib3_logger.setLevel(logging.CRITICAL) # suppress messages upon download

In [14]:
from google.cloud import storage

storage_client = storage.Client(project = project_name)
#bucket = storage_client.get_bucket(bucket_name)

series_instance_uids = []

#blobs = storage_client.list_blobs(bucket)
blobs = storage_client.list_blobs(bucket_name, prefix=bucket_path) #, delimiter='/') #don't search in non-axial sub-folder
for blob in blobs:
    bn = blob.name 
    # ex: bpr-results/1.2.826.0.1.3680043.10.474.419639.105799060738901793068313281334.json
    bs = bn.split('.')
    if bs[-1] == 'json':
      #print(bn)
      bss = bn.split('/')[1]
      #print(bss)
      bs3 = bss.split('.')
      bs4 = '.'.join(bs3[:-1])
      series_instance_uids.append(bs4)

print(series_instance_uids[:10])

['1.2.826.0.1.3680043.10.474.419639.105799060738901793068313281334', '1.2.826.0.1.3680043.10.474.419639.106364025147079899289440200715', '1.2.826.0.1.3680043.10.474.419639.149051607502633615235577977952', '1.2.826.0.1.3680043.10.474.419639.189346812051260775638656981947', '1.2.826.0.1.3680043.10.474.419639.192916356998524553834723357563', '1.2.826.0.1.3680043.10.474.419639.198735019931123383691750806063', '1.2.826.0.1.3680043.10.474.419639.249044315484665760654506668895', '1.2.826.0.1.3680043.10.474.419639.259584978733948574762940092562', '1.2.826.0.1.3680043.10.474.419639.269534881585852761235289087419', '1.2.826.0.1.3680043.10.474.419639.272117657178318732150423166273']


In [ ]:
print(len(series_instance_uids))

1196


In [15]:
from google.cloud import storage

storage_client = storage.Client(project = project_name)
#bucket = storage_client.get_bucket(bucket_name)

series_instance_uids = []

#blobs = storage_client.list_blobs(bucket)
blobs = storage_client.list_blobs(bucket_name, prefix=bucket_path, delimiter='/')
for blob in blobs:
    bn = blob.name 
    # ex: bpr-results/1.2.826.0.1.3680043.10.474.419639.105799060738901793068313281334.json
    (r, ext) = os.path.splitext(bn)
    #print(r,ext)
    if ext == '.json':
      u = os.path.split(r)[-1]
      #print(u)
      series_instance_uids.append(u)

print(series_instance_uids[:10])

['1.2.826.0.1.3680043.10.474.419639.105799060738901793068313281334', '1.2.826.0.1.3680043.10.474.419639.106364025147079899289440200715', '1.2.826.0.1.3680043.10.474.419639.149051607502633615235577977952', '1.2.826.0.1.3680043.10.474.419639.189346812051260775638656981947', '1.2.826.0.1.3680043.10.474.419639.192916356998524553834723357563', '1.2.826.0.1.3680043.10.474.419639.198735019931123383691750806063', '1.2.826.0.1.3680043.10.474.419639.249044315484665760654506668895', '1.2.826.0.1.3680043.10.474.419639.259584978733948574762940092562', '1.2.826.0.1.3680043.10.474.419639.269534881585852761235289087419', '1.2.826.0.1.3680043.10.474.419639.272117657178318732150423166273']


In [ ]:
print(len(series_instance_uids))

495


In [16]:
df = ct_limited_open_a1_r1
k=0

good_seriesuids = []

#series_instance_uids = ['1.2.826.0.1.3680043.10.474.419639.106364025147079899289440200715']

for seriesuid in series_instance_uids:
  #print(seriesuid)
  studyuids = list(set(df[df['SeriesInstanceUID']==seriesuid]['StudyInstanceUID'].tolist()))
  if len(studyuids)>0:
    k=k+1
    #print(seriesuid)
    studyuid = studyuids[0]
    sops = df[ (df['StudyInstanceUID']==studyuid) & (df['SeriesInstanceUID']==seriesuid) ]['SOPInstanceUID'].tolist()
    print(len(sops))
    #download_series(studyuid, seriesuid, sops, path_downloaded)
    good_seriesuids.append(seriesuid)
print(k)

178
180
145
193
111
184
152
262
194
272
152
206
271
194
114
658
106
233
997
466
209
214
427
915
172
735
262
194
115
133
133
165
704
189
199
199
202
252
118
153
122
171
165
704
165
707
170
143
158
158
301
243
123
123
107
187
132
132
115
229
103
205
163
144
141
186
118
232
265
162
115
119
131
103
186
165
140
163
237
160
131
146
199
199
365
158
137
185
201
152
163
194
208
178
217
116
113
139
137
280
163
156
132
233
124
229
113
199
149
175
143
107
175
123
207
164
114
105
157
157
130
162
126
116
194
120
149
599
145
170
137
232
141
159
179
143
167
143
135
111
321
178
171
218
190
209
319
122
150
140
212
164
230
182
200
104
143
151
301
101
178
151
179
156
209
115
172
200
219
158
204
184
233
118
168
239
126
128
136
204
140
273
176
127
149
207
128
158
146
139
164
163
168
205
135
135
140
146
173
113
156
125
139
119
216
599
141
125
123
193
200
146
153
293
194
217
201
123
261
114
107
199
145
185
103
218
189
205
108
178
115
171
120
162
189
403
229
188
108
118
194
283
200
115
143
218
136
153
130
114


In [17]:
!pip uninstall highdicom
!git clone https://github.com/herrmannlab/highdicom.git
#!cd highdicom && python setup.py install
!cd highdicom && pip install .

Cloning into 'highdicom'...
remote: Enumerating objects: 5612, done.
remote: Counting objects: 100% (2165/2165), done.
remote: Compressing objects: 100% (548/548), done.
remote: Total 5612 (delta 1847), reused 1719 (delta 1545), pack-reused 3447
Receiving objects: 100% (5612/5612), 3.08 MiB | 9.27 MiB/s, done.
Resolving deltas: 100% (3650/3650), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/highdicom
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 339 kB 5.1 MB/s 
  Created wheel for highdicom: filename=highdicom-0.19.0-py3-non

In [18]:
# Packages for the structured report 

import highdicom

from pathlib import Path

import highdicom as hd

from pydicom.uid import generate_uid
from pydicom.filereader import dcmread
from pydicom.sr.codedict import codes

from highdicom.sr.content import (
    FindingSite,
    ImageRegion,
    ImageRegion3D,
    SourceImageForRegion,
    SourceImageForMeasurement,
    SourceImageForMeasurementGroup
)
from highdicom.sr.enum import GraphicTypeValues3D
from highdicom.sr.enum import GraphicTypeValues
from highdicom.sr.sop import Comprehensive3DSR, ComprehensiveSR
from highdicom.sr.templates import (
    DeviceObserverIdentifyingAttributes,
    Measurement,
    MeasurementProperties,
    MeasurementReport,
    MeasurementsAndQualitativeEvaluations,
    ObservationContext,
    ObserverContext,
    PersonObserverIdentifyingAttributes,
    PlanarROIMeasurementsAndQualitativeEvaluations,
    RelationshipTypeValues,
    TrackingIdentifier,
    QualitativeEvaluation,
    ImageLibrary,
    ImageLibraryEntryDescriptors
)
from highdicom.sr.value_types import (
    CodedConcept,
    CodeContentItem,
)

import logging
logger = logging.getLogger("highdicom.sr.sop")
logger.setLevel(logging.INFO)

SR regions

In [19]:
# takes as input a json file and slice_index, returns the regions assigned to the slice 
def convert_slice_to_region(bpr_data, slice_index):

  """ 
  Given the slice_index, this returns a list of corresponding regions that match

  Inputs: 
    bpr_data    : a dictionary, where for each of the six regions, a list of 
                  slice indices are provided 
    slice_index : slice number you want to obtain the list of classified regions 
                  for
  Returns
    regions     : list of regions for that slice_index

  """

  # find where the slice_index appears across all regions
  # get the names of the regions

  num_regions = len(bpr_data)
  region_names = list(bpr_data.keys())
  regions = [] 

  for n in range(0,num_regions):
    vals = bpr_data[region_names[n]]
    if slice_index in vals:
      regions.append(region_names[n])

  return regions

In [20]:
def create_structured_report_for_body_part_regression_regions(files, 
                                                              json_file, 
                                                              output_SR_file, 
                                                              bpr_revision_number,
                                                              bpr_regions_df):

  """Takes as input a set of DICOM files and the corresponding body part regression json file, 
     and writes a structured report (SR) to disk
     
  Inputs: 
    files               : list of CT dicom files 
    json_file           : the json file created from the BodyPartRegression prediction
    output_SR_file      : output filename for the structured report 
    bpr_revision_number : specific revision number of the bpr repo 
    bpr_regions_df      : holds the metadata needed for the bpr target regions 

  Outputs:
    writes the SR out to the output_SR_file.    
     
  """


  # ------ order the CT files according to the ImagePositionPatient and ImageOrientation ----# 

  num_files = len(files)
  # print ("num_files: " + str(num_files))

  pos_all = []  
  sop_all = [] 

  for n in range(0,num_files):
    # read dcm file 
    filename = files[n]
    ds = dcmread(filename)
    # print(ds)

    # get ImageOrientation (0020, 0037)
    # print(ds['0x0020','0x0037'].value)
    ImageOrientation = ds['0x0020','0x0037'].value

    # get ImagePositionPatient (0020, 0032) 
    ImagePositionPatient = ds['0x0020','0x0032'].value

    # calculate z value
    x_vector = ImageOrientation[0:3]
    y_vector = ImageOrientation[3:]
    z_vector = np.cross(x_vector,y_vector)

    # multiple z_vector by ImagePositionPatient
    pos = np.dot(z_vector,ImagePositionPatient)
    pos_all.append(pos)

    # get the SOPInstanceUID 
    sop = ds['0x0008', '0x0018'].value
    sop_all.append(sop)

#----- order the SOPInstanceUID/files by z value ----# 

  sorted_ind = np.argsort(pos_all)
  pos_all_sorted = np.array(pos_all)[sorted_ind.astype(int)]
  sop_all_sorted = np.array(sop_all)[sorted_ind.astype(int)]
  files_sorted = np.array(files)[sorted_ind.astype(int)]

  #---- Open the json file and parse the list of regions per slice -----# 

  f = open(json_file)
  json_data = json.load(f)
  bpr_data = json_data['body part examined']

  # return a list where each entry is per slice and is a array of possible regions 
  bpr_slice_scores = json_data['cleaned slice scores']
  num_slices = len(bpr_slice_scores)

  num_regions = len(bpr_data)
  regions = [] 

  # print('num_slices: ' + str(num_slices))

  for slice_index in range(0,num_slices):
    region = convert_slice_to_region(bpr_data, slice_index)
    regions.append(region)

  # ----- Create the structured report ----- # 

  # Create the report content

  procedure_code = CodedConcept(value="363679005", scheme_designator="SCT", 
                                meaning="Imaging procedure")

  # Describe the context of reported observations: the person that reported
  # the observations and the device that was used to make the observations
  observer_person_context = ObserverContext(
      observer_type=codes.DCM.Person,
      observer_identifying_attributes=PersonObserverIdentifyingAttributes(
          name='Anonymous^Reader'
      )
  )
  # observer_device_context = ObserverContext(
  #     observer_type=codes.DCM.Device,
  #     observer_identifying_attributes=DeviceObserverIdentifyingAttributes(
  #         uid=generate_uid(), name="BodyPartRegression"
  #     )
  observer_device_context = ObserverContext(
    observer_type=codes.DCM.Device,
    observer_identifying_attributes=DeviceObserverIdentifyingAttributes(
        uid=generate_uid(), name="BodyPartRegression", 
        model_name = bpr_revision_number
    )
  )
  observation_context = ObservationContext(
      #observer_person_context=observer_person_context,
      observer_device_context=observer_device_context,
  )

  imaging_measurements = []
  evidence = []

  tracking_uid = generate_uid()

  qualitative_evaluations = []

  print('num_slices: ' + str(num_slices))

#----------- Per slice ---------#

  for n in range(0,num_slices):

    slice_region = regions[n]

    # qualitative_evaluations = convert_regions_list_to_qualitative_evaluations(slice_region)

    # ----- per region ---- # 
    qualitative_evaluations = [] 
    # num_regions = len(regions)
    num_regions = len(slice_region)

    # for region in regions: 
    for region in slice_region: 
      row = bpr_regions_df.loc[bpr_regions_df['BPR_code_region'] == region]
      qualitative_evaluations.append(
          QualitativeEvaluation(
              CodedConcept(
                            value=str(row["target_CodeValue"].values[0]),
                            meaning=str(row["target_CodeMeaning"].values[0]).replace(u'\xa0', u' '),
                            # meaning = "Target Region",
                            scheme_designator=str(row["target_CodingSchemeDesignator"].values[0])
                            ), 
              CodedConcept(
                            value=str(row["CodeValue"].values[0]),
                            meaning=str(row["CodeMeaning"].values[0]),
                            scheme_designator=str(row["CodingSchemeDesignator"].values[0])
                          )
              )
          )

    # In the correct order 
    reference_dcm_file = files_sorted[n]
    image_dataset = dcmread(reference_dcm_file)
    evidence.append(image_dataset)

    # NS
    print(evidence[0])

    src_image = hd.sr.content.SourceImageForMeasurementGroup.from_source_image(image_dataset)

    # tracking_id = "Annotations group x"
    tracking_id = "Annotations group " + str(n+1) # start indexing with 1

    measurements_group = MeasurementsAndQualitativeEvaluations(
                  tracking_identifier=TrackingIdentifier(
                      uid=tracking_uid,
                      identifier=tracking_id
                  ),
                  qualitative_evaluations=qualitative_evaluations,
                  source_images=[src_image]
              )




    imaging_measurements.append(
      measurements_group
            )
    
  #-------------------#
    
  measurement_report = MeasurementReport(
      observation_context=observation_context,
      procedure_reported=procedure_code,
      imaging_measurements=imaging_measurements
  )

  # Create the Structured Report instance
  series_instance_uid = generate_uid()
  sr_dataset = Comprehensive3DSR(
      evidence=evidence,
      content=measurement_report[0],
      series_number=100,
      series_instance_uid=series_instance_uid,
      sop_instance_uid=generate_uid(),
      instance_number=1,
      manufacturer='MIDRC',
      is_complete = True,
      is_final=True,
      series_description='BPR region annotations'
  )
  


  pydicom.write_file(output_SR_file, sr_dataset)

  return sr_dataset

SR landmarks

In [21]:
def load_json(json_file):
  f = open(json_file)
  json_data = json.load(f)
  return json_data

In [26]:
def crop_scores(scores, start_score, end_score):
    scores = np.array(scores)
    min_scores = np.where(scores < start_score)[0]
    max_scores = np.where(scores > end_score)[0]

    min_index = 0
    max_index = len(scores)

    if len(min_scores) > 0:
        min_index = np.nanmax(min_scores)

    if len(max_scores) > 0:
        max_index = np.nanmin(max_scores)

    return min_index, max_index

In [22]:
def get_indices_from_json(filename, tag_start, tag_end):

  """
  Gets the indices of the particular anatomy specified by the tag_start and 
  tag_end for a particular patient. 

  Arguments:
    filename  : required - the patient json filename 
    tag_start : required - the string for the start of the anatomical region 
    tag_end   : required - the string for the end of the anatomical region 

  Outputs:
    min_index : the minimum index in the patient coordinate system
    max_index : the maximum index in the patient coordinate system 
  """

  # These scores are the same for all patients  
  x = load_json(filename)

  start_score = x["look-up table"][tag_start]["mean"]
  end_score = x["look-up table"][tag_end]["mean"]

  # The actual indices 
  min_index, max_index = crop_scores(x["cleaned slice scores"], start_score, end_score)

  return min_index, max_index

In [23]:
def get_landmark_indices_from_json(filename):

  """
  Gets the indices of each landmark in the patient json filename and converts 
  it to slice indices. 

  Arguments:
    filename  : required - the patient json filename 

  Outputs:
    list of the corresponding landmark slices in the space of the patient. 
    Landmarks at the extreme ends - most inferior and most superior axial slices
    are removed. 
  """

  # Get the cleaned slice scores to determine the number of slices 
  x = load_json(filename)
  num_slices = len(x["cleaned slice scores"])

  # Get the list of landmarks 
  landmarks = list(x["look-up table"].keys())
  num_landmarks = len(landmarks)

  # Get the expected z_spacing - if less than 0, slices are in reverse order 
  valid_z_spacing = x["valid z-spacing"]

  # Get values for all tags 
  # Reorder the landmarks according to the mean values in ascending order 
  landmarks_dict_sorted = {}
  for n in range(0,num_landmarks):
    landmark = landmarks[n]
    landmarks_dict_sorted[landmark] = x["look-up table"][landmark]['mean']
  landmark_dict_sorted = dict(sorted(landmarks_dict_sorted.items(), key=lambda item: item[1]))
  landmarks = list(landmark_dict_sorted.keys())

  # Calculate the actual slice indices of each landmark 
  landmark_indices = {}
  for n in range(0,num_landmarks):
    landmark = landmarks[n]
    score = landmark_dict_sorted[landmark] # x["look-up table"][landmark]["mean"]
    min_index, max_index = crop_scores(x["cleaned slice scores"], score, score)
    if (valid_z_spacing > 0): 
      landmark_indices[landmark] = min_index 
    else: 
      landmark_indices[landmark] = num_slices - min_index 

  # Programmatically remove values from dictionary if it is at most inferior
  # or most superior slice 
  for n in range(0,num_landmarks):
    landmark = landmarks[n]
    if (landmark_indices[landmark]==0):
      del landmark_indices[landmark]
    elif (landmark_indices[landmark]==num_slices-1):
      del landmark_indices[landmark]

  return landmark_indices

In [24]:
def get_landmark_indices_list_in_slice(landmark_indices, slice_number):

  """
  Gets the list of landmarks that are assigned to a particular slice number.

  Arguments:
    flandmark_indices  : the dictionary holding the slice number for each 
                         landmark
    slice_number       : the particular slice to obtain the list of landmarks
                         for

  Outputs:
    list of the landmarks that correspond to the slice_number
  """

  key_list = list()
  items_list = landmark_indices.items()
  for item in items_list:
    if item[1] == slice_number:
        key_list.append(item[0])

  return key_list

def convert_landmarks_list_to_qualitative_evaluations(landmark_list,
                                                      landmarks_df):

  """
  Converts the list of landmarks to a qualitative_evaluations for the 
  structured report. 

  Arguments:
    landmark_list  : list of landmarks that correspond to a particular slice
    landmarks_df   : the dataframe holding the bpr landmarks metadata needed to
                     create the structured report

  Outputs:
    list of QualitativeEvaluation
  """


  qualitative_evaluations = [] 
  num_landmarks_in_slice = len(landmark_list)
  for landmark in landmark_list: 
    if not landmark in landmarks_df["BPR_code"].values:
      print("ERROR: Failed to map BPR landmark "+landmark)
      break
    else:
      landmark_row = landmarks_df[landmarks_df["BPR_code"] == landmark]
      # landmark_code = CodedConcept(value = str(int(landmark_row["CodeValue"].values[0])),
      #                              meaning = landmark_row["CodeMeaning"].values[0],
      #                              scheme_designator = landmark_row["CodingSchemeDesignator"].values[0])
      landmark_code = CodedConcept(value = str(landmark_row["CodeValue"].values[0].astype(np.int64)),
                                meaning = str(landmark_row["CodeMeaning"].values[0]),
                                scheme_designator = str(landmark_row["CodingSchemeDesignator"].values[0]))
      #print(landmarks_df["CodingSchemeDesignator"].values[0])
      landmark_modifier_code = None
      if not pd.isna(landmarks_df["modifier_CodeValue"].values[0]):
        # landmark_modifier_code = CodedConcept(value = str(int(landmark_row["modifier_CodeValue"].values[0])),
        #                              meaning = landmark_row["modifier_CodeMeaning"].values[0],
        #                              scheme_designator = landmark_row["modifier_CodingSchemeDesignator"].values[0])
        landmark_modifier_code = CodedConcept(value = str(landmark_row["modifier_CodeValue"].values[0].astype(np.int64)),
                                meaning = str(landmark_row["modifier_CodeMeaning"].values[0]),
                                scheme_designator = str(landmark_row["modifier_CodingSchemeDesignator"].values[0]))
        #print(landmarks_df["modifier_CodingSchemeDesignator"].values[0])
        qual = QualitativeEvaluation(CodedConcept(
                value="123014",
                meaning="Target Region",
                scheme_designator="DCM"  
                ), 
                landmark_code
              )
    
    if landmark_modifier_code is not None:
      qual_modifier = CodeContentItem(
          name=CodedConcept(
              value='106233006',
              meaning='Topographical modifier',
              scheme_designator='SCT',
          ),
          value=landmark_modifier_code,
          relationship_type=RelationshipTypeValues.HAS_CONCEPT_MOD
      )
      qual.append(qual_modifier)

    qualitative_evaluations.append(qual)

  return qualitative_evaluations


In [25]:
def create_structured_report_for_body_part_regression_landmarks(files, 
                                                                json_file, 
                                                                output_SR_file, 
                                                                bpr_revision_number,
                                                                landmarks_df):

  """Takes as input a set of DICOM files and the corresponding body part regression json file, 
     and writes a structured report (SR) to disk
     
  Inputs: 
    files               : list of CT dicom files 
    json_file           : the json file created from the BodyPartRegression prediction
    output_SR_file      : output filename for the structured report 
    bpr_revision_number : specific revision number of the bpr repo 
    landmarks_df        : the dataframe holding the bpr landmarks metadata needed to
                          create the structured report

  Outputs:
    writes the SR out to the output_SR_file.    
     
  """


  # ------ order the CT files according to the ImagePositionPatient and ImageOrientation ----# 

  num_files = len(files)

  pos_all = []  
  sop_all = [] 

  for n in range(0,num_files):
    # read dcm file 
    filename = files[n]
    ds = dcmread(filename)
    # print(ds)

    # get ImageOrientation (0020, 0037)
    # ImageOrientation = ds['0x0020','0x0037'].value
    ImageOrientation = ds.ImageOrientationPatient
    #ImageOrientation = ds.ImageOrientationPatient.value

    # get ImagePositionPatient (0020, 0032) 
    # ImagePositionPatient = ds['0x0020','0x0032'].value
    ImagePositionPatient = ds.ImagePositionPatient

    # calculate z value
    x_vector = ImageOrientation[0:3]
    y_vector = ImageOrientation[3:]
    z_vector = np.cross(x_vector,y_vector)

    # multiple z_vector by ImagePositionPatient
    pos = np.dot(z_vector,ImagePositionPatient)
    pos_all.append(pos)

    # get the SOPInstanceUID 
    sop = ds['0x0008', '0x0018'].value
    sop_all.append(sop)

    #----- order the SOPInstanceUID/files by z value ----# 

  sorted_ind = np.argsort(pos_all)
  pos_all_sorted = np.array(pos_all)[sorted_ind.astype(int)]
  sop_all_sorted = np.array(sop_all)[sorted_ind.astype(int)]
  files_sorted = np.array(files)[sorted_ind.astype(int)]

  #----- Get the landmarks as indices -----# 

  landmark_indices = get_landmark_indices_from_json(json_file)

  # ----- Create the structured report ----- # 

  # Create the report content

  procedure_code = CodedConcept(value="363679005", scheme_designator="SCT", 
                                meaning="Imaging procedure")

  # Describe the context of reported observations: the person that reported
  # the observations and the device that was used to make the observations
  # observer_person_context = ObserverContext(
  #     observer_type=codes.DCM.Person,
  #     observer_identifying_attributes=PersonObserverIdentifyingAttributes(
  #         name='Anonymous^Reader'
  #     )
  # )

  # observer_device_context = ObserverContext(
  #     observer_type=codes.DCM.Device,
  #     observer_identifying_attributes=DeviceObserverIdentifyingAttributes(
  #         uid=generate_uid(), name="BodyPartRegression_landmarks"
  #     )
  # )

  observer_device_context = ObserverContext(
      observer_type=codes.DCM.Device,
      observer_identifying_attributes=DeviceObserverIdentifyingAttributes(
          uid=generate_uid(), name="BodyPartRegression_landmarks", 
          model_name = bpr_revision_number
      )
  )
  # inputMetadata["observerContext"] = {
  #                   "ObserverType": "DEVICE",
  #                   "DeviceObserverName": "pyradiomics",
  #                   "DeviceObserverModelName": "v3.0.1"
  #                 }

  observation_context = ObservationContext(
      #observer_person_context=observer_person_context,
      observer_device_context=observer_device_context,
  )

  imaging_measurements = []
  evidence = []

  qualitative_evaluations = []

  tracking_uid = generate_uid()

  #------------- Per slice - only include landmarks that exist ------# 

  num_slices = len(files_sorted)
  annotation_count = 1 

  for n in range(0,num_slices):

    # find all the dictionary entries that match this slice number - returns a list of landmarks 
    landmark_indices_list = get_landmark_indices_list_in_slice(landmark_indices, n) # n = slice number 

    # Only include if there is a landmark for a slice 
    if (landmark_indices_list):
    
      # Create QualitativeEvaluations
      qualitative_evaluations = convert_landmarks_list_to_qualitative_evaluations(landmark_indices_list,
                                                                                  landmarks_df)

      # In the correct order 
      reference_dcm_file = files_sorted[n]
      image_dataset = dcmread(reference_dcm_file)
      evidence.append(image_dataset)

      src_image = hd.sr.content.SourceImageForMeasurementGroup.from_source_image(image_dataset)

      # tracking_id = "Annotations group x"
      # tracking_id = "Annotations group landmarks"
      # tracking_id = "Annotations group landmarks " + str(n+1) # start indexing with 1
      tracking_id = "Annotations group landmarks " + str(annotation_count) # start indexing with 1

      measurements_group = MeasurementsAndQualitativeEvaluations(
                    tracking_identifier=TrackingIdentifier(
                        uid=tracking_uid,
                        identifier=tracking_id
                    ),
                    qualitative_evaluations=qualitative_evaluations,
                    source_images=[src_image]
                )




      imaging_measurements.append(
        measurements_group
              )
      
      annotation_count += 1 # keep track of number of annotations

       #-------------------#
    
  measurement_report = MeasurementReport(
      observation_context=observation_context,
      procedure_reported=procedure_code,
      imaging_measurements=imaging_measurements
  )

  # Create the Structured Report instance
  series_instance_uid = generate_uid()
  sr_dataset = Comprehensive3DSR(
      evidence=evidence,
      content=measurement_report[0],
      series_number=101, # was 100 for regions
      series_instance_uid=series_instance_uid,
      sop_instance_uid=generate_uid(),
      instance_number=1,
      manufacturer='IDC',
      is_complete = True,
      is_final=True,
      series_description='BPR landmark annotations'
  )
  # series_description='BPR landmark annotations'

  pydicom.write_file(output_SR_file, sr_dataset)

  return sr_dataset

In [27]:
#upload files:
# - bpr_regions_code_mapping
# - bpr_landmarks_code_mapping

from google.colab import files
uploaded = files.upload()

Saving bpr_landmarks_code_mapping.csv to bpr_landmarks_code_mapping.csv
Saving bpr_regions_code_mapping.csv to bpr_regions_code_mapping.csv


In [28]:
# Processing series

path_downloaded = '/content/downloaded_data'
path_nifti = '/content/nifti_data'
path_json =  '/content/json_data'
path_nrrd = '/content/nrrd_data'
path_sr = '/content/sr_data'

df = ct_limited_open_a1_r1

good_seriesuids = ['1.2.826.0.1.3680043.10.474.419639.149051607502633615235577977952']

for seriesuid in good_seriesuids:

  # clean up from previous iterations and recreate temp directories
  if 1:
    for x in [path_downloaded, path_nifti, path_json, path_nrrd]:
      if os.path.isdir(x):
        try:
          shutil.rmtree(x)
        except OSError as err:
          print("Error: %s : %s" % (x, err.strerror))  
      os.mkdir(x)

      studyuids = list(set(df[df['SeriesInstanceUID']==seriesuid]['StudyInstanceUID'].tolist()))
  if len(studyuids)>0:
    print(seriesuid)
  studyuid = studyuids[0]
  sops = df[ (df['StudyInstanceUID']==studyuid) & (df['SeriesInstanceUID']==seriesuid) ]['SOPInstanceUID'].tolist()
  print(len(sops))

  download_series(studyuid, seriesuid, sops, path_downloaded)

  file_gs_uri_json = "gs://%s/%s%s.json" % (bucket_name, bucket_path, seriesuid)
  
  cmd1 = "gsutil cp %s %s" % (file_gs_uri_json, path_json)
  print(cmd1)
  os.system(cmd1)

  dcm_input_list = glob.glob('/content/downloaded_data/*.dcm')
  #print(dcm_input_list)
  
  dcm_input_list = glob.glob('/content/downloaded_data/*.dcm')
  fn_json = os.path.join(path_json, '%s.json' % seriesuid)
  fn_sr_regions = os.path.join(path_sr, 'sr_regions_%s.dcm' % seriesuid)
  df_regions = pd.read_csv('/content/bpr_regions_code_mapping.csv')
  df_landmarks = pd.read_csv('/content/bpr_landmarks_code_mapping.csv')

1.2.826.0.1.3680043.10.474.419639.149051607502633615235577977952
145
gsutil cp gs://midrc-analysis-bwh/bpr-results/1.2.826.0.1.3680043.10.474.419639.149051607502633615235577977952.json /content/json_data


In [29]:
create_structured_report_for_body_part_regression_regions(dcm_input_list, 
                                                              fn_json, 
                                                              fn_sr_regions, 
                                                              bpr_revision_number,
                                                              df_regions)

Streaming output truncated to the last 5000 lines.
Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 198
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: CT Image Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 1.2.826.0.1.3680043.10.474.419639.141864085480034350300530809396
(0002, 0010) Transfer Syntax UID                 UI: JPEG Lossless, Non-Hierarchical, First-Order Prediction (Process 14 [Selection Value 1])
(0002, 0012) Implementation Class UID            UI: 1.2.40.0.13.1.1.1
(0002, 0013) Implementation Version Name         SH: 'dcm4che-1.4-JP'
-------------------------------------------------
(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'AXIAL']
(0008, 0016) SOP Class UID                       UI: CT Image Storage
(0008, 0018) SOP Instance UID     

AttributeError: ignored

In [30]:
create_structured_report_for_body_part_regression_landmarks(dcm_input_list, 
                                                              fn_json, 
                                                              fn_sr_regions, 
                                                              bpr_revision_number,
                                                              df_landmarks)

AttributeError: ignored